<a href="https://www.kaggle.com/code/yorkyong/aspect-based-sentiment-analysis-with-llama-3-1?scriptVersionId=200408281" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
#with reference from https://www.kaggle.com/code/lucamassaron/fine-tune-llama-2-for-sentiment-analysis

In [2]:
!pip install -q -U torch --index-url https://download.pytorch.org/whl/cu117
!pip install -q -U -i https://pypi.org/simple/ bitsandbytes
!pip install -q -U transformers
!pip install -q -U accelerate
!pip install -q -U datasets
!pip install -q -U trl
!pip install -q -U peft

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "true"

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
import datetime as dt
import re
import os
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
from peft import LoraConfig, PeftConfig
from trl import SFTTrainer
from trl import setup_chat_format
from transformers import (AutoConfig,
                          AutoModelForCausalLM, 
                          AutoTokenizer,
                          LlamaForCausalLM, 
                          LlamaTokenizer,
                          BitsAndBytesConfig, 
                          TrainingArguments, 
                          pipeline, 
                          logging)
from sklearn.metrics import (accuracy_score, 
                             classification_report, 
                             confusion_matrix)
from sklearn.model_selection import train_test_split

In [6]:
pd.set_option('display.max_colwidth', None)  # Show full width of each column

In [7]:
print(f"pytorch version {torch.__version__}")

pytorch version 2.4.0


In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"working on {device}")

working on cuda:0


# Preparing the data and the core evaluation functions

In [9]:
filename= '/kaggle/input/mcdonalds-review-dataset-6-months-clean-sentiment/cleaned_data _for_ABSA.xlsx'

In [10]:
df = pd.read_excel(filename)

In [11]:
df.head()

,store_address,location,reviewer_id,review_time,actual_sentiment,review,token_review_clean
0,"1044 US-11, Champlain, NY 12919, United States",12919,11931,6 months ago,negative,Food seems to be getting worse. Tables dirty. Sketchy crowd loitering within the store.,food seems getting worse table dirty sketchy crowd loitering within store
1,"1044 US-11, Champlain, NY 12919, United States",12919,11895,a week ago,negative,Food was slow. Manager (short hair woman) was having a mental breakdown or something and was being rude. She needs to be fired. Her name was Elizabeth or something. Very unprofessional.,food slow manager short hair woman mental breakdown something rude need fired name elizabeth something unprofessional
2,"1044 US-11, Champlain, NY 12919, United States",12919,11920,a month ago,negative,Franchise owners need to hire more staff to keep up with demand,franchise owner need hire staff keep demand
3,"1044 US-11, Champlain, NY 12919, United States",12919,11912,2 weeks ago,negative,Haven't seen a McDonald's so lost in my life :(,seen lost life
4,"1044 US-11, Champlain, NY 12919, United States",12919,11897,4 months ago,negative,"I came there to pick up order around 5 in the morning, there was a manager(brown hair) who dealt with me very rudely, i politely requested for my uber pick up but he showed me attitude and just shut the window. That was an awful experience for me and i would like to suggest them to show some etiquettes and respect towards their customers. I also work in a restaurant and know how to deal with the customers and uber orders.",came pick order around morning managerbrown hair dealt rudely politely requested uber pick showed attitude shut window awful experience would like suggest show etiquette respect towards customer also work restaurant know deal customer uber order


# Prompt

In [12]:
def generate_prompt(review):
    return f"""
Analyze the sentiment of the following review for five specific aspects: service, cleanliness, food, price, and others.
For each aspect, assign a sentiment score based on the following scale:
0: Aspect is not mentioned
1: Negative sentiment
2: Neutral sentiment
3: Positive sentiment
IMPORTANT:
- If the review is too brief or generic and not about specific aspects, assess the overall sentiment and assign it to others.
- Only provide scores for aspects that are EXPLICITLY mentioned or can be DIRECTLY inferred from the review.
- If an aspect is not mentioned or cannot be directly inferred, assign it a score of 0.
- Use the 'others' category to capture any significant aspects mentioned that don't fit into service, cleanliness, food, or price.
- Do NOT repeat the review or provide any additional analysis or explanations beyond the scores.
- Provide the scores ONLY ONCE in the exact format shown below.
Provide your analysis in the following format ONLY:
service=[score]
cleanliness=[score]
food=[score]
price=[score]
others=[score]
Review: {review}
Analysis:
"""

In [21]:
# # Generate the full prompt (input) and retain sentiment as a separate column
# df['prompt'] = df.apply(generate_prompt, axis=1)

# Now convert the DataFrame to a dataset
df_data = Dataset.from_pandas(df)

In [23]:
df_data

Dataset({
    features: ['store_address', 'location', 'reviewer_id', 'review_time', 'actual_sentiment', 'review', 'token_review_clean'],
    num_rows: 2532
})

# Testing model without fine-tuning

In [15]:
# # For clearing GPU memory
# import gc

# del [
#     model, 
#     tokenizer, 
# ]
# # del [df, X_train, X_eval]

In [16]:
# for _ in range(100):
#     torch.cuda.empty_cache()
#     gc.collect()

In [17]:
model_name = "/kaggle/input/llama-3.1/transformers/8b-instruct/2"
# model_name = "/kaggle/input/llama-3.2/transformers/3b-instruct/1"
# model_name = "/kaggle/input/llama-3.1/transformers/405b-instruct/1"

compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, 
    bnb_4bit_quant_type="nf4", 
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map=device,
    torch_dtype=compute_dtype,
    quantization_config=bnb_config, 
)

model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_name, 
                                          trust_remote_code=True,
                                         )
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model, tokenizer = setup_chat_format(model, tokenizer)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [18]:
print(len(tokenizer))  # Check vocabulary size

128258


In [24]:
def predict(df_data, model, tokenizer, debug=False):
    y_pred = []
    aspects = ['service', 'cleanliness', 'food', 'price', 'others']
    
    for i in tqdm(range(len(df_data))):
        review = df_data[i]['review']
        prompt = generate_prompt(review)
        
        pipe = pipeline(task="text-generation", 
                        model=model, 
                        tokenizer=tokenizer, 
                        max_new_tokens=21, 
                        temperature=0.07,
                       )
        try:
            result = pipe(prompt)
            generated_text = result[0]['generated_text']
            
            # Extract only the part that matches our expected format
            answer = re.search(r'Analysis:[\s\S]*', generated_text)
            if answer:
                answer = answer.group(0)
            else:
                raise ValueError("Unexpected output format")
            
            aspect_sentiments = {}
            
            # Updated regex to capture scores more accurately, even with partial output
            matches = re.findall(r'(\w+)=\[?(\d)\]?', answer)
            
            for match in matches:
                aspect, score = match
                aspect = aspect.lower()
                score = int(score)
                if aspect in aspects and score in [0, 1, 2, 3]:
                    aspect_sentiments[aspect] = score
            
            # If no aspects were found or all aspects have zero scores, it's treated as a brief/generic review
            if not aspect_sentiments or all(score == 0 for score in aspect_sentiments.values()):
                aspect_sentiments = {aspect: 0 for aspect in aspects}
            
            # Fill in any missing aspects with 0
            for aspect in aspects:
                if aspect not in aspect_sentiments:
                    aspect_sentiments[aspect] = 0
            
            y_pred.append(aspect_sentiments)
            
            if debug:
                print(f"\nItem {i}:")
                print(f"Review: {review}")
                print(f"Generated: {answer}")
                print(f"Parsed sentiments: {aspect_sentiments}")
        
        except Exception as e:
            print(f"Error processing item {i}: {str(e)}")
            y_pred.append({aspect: 0 for aspect in aspects})
    
    return y_pred

In [25]:
y_pred = predict(df_data, model, tokenizer, debug=True)

  0%|          | 1/2532 [00:03<2:08:12,  3.04s/it]


Item 0:
Review: Food seems to be getting worse. Tables dirty. Sketchy crowd loitering within the store.
Generated: Analysis:
service=[1]
cleanliness=[1]
food=[1]
price=[0]
others=[1]
Parsed sentiments: {'service': 1, 'cleanliness': 1, 'food': 1, 'price': 0, 'others': 1}


  0%|          | 2/2532 [00:05<1:55:50,  2.75s/it]


Item 1:
Review: Food was slow. Manager (short hair woman) was having a mental breakdown or something and was being rude. She needs to be fired. Her name was Elizabeth or something. Very unprofessional.
Generated: Analysis:
service=[1]
cleanliness=[0]
food=[1]
price=[0]
others=[1]
Parsed sentiments: {'service': 1, 'cleanliness': 0, 'food': 1, 'price': 0, 'others': 1}


  0%|          | 3/2532 [00:07<1:49:07,  2.59s/it]

...


Item 2529:
Review: security stop notch
Generated: Analysis:
service=[score]
cleanliness=[score]
food=[score]
price=[score]
others=[score]
Parsed sentiments: {'service': 0, 'cleanliness': 0, 'food': 0, 'price': 0, 'others': 0}


100%|█████████▉| 2531/2532 [1:44:09<00:02,  2.42s/it]


Item 2530:
Review: They charge more for the same food because it's on the strip. I never understood that. But the food and the service were great at the drive thru!
Generated: Analysis:
service=3
cleanliness=0
food=3
price=1
others=0


Parsed sentiments: {'service': 3, 'cleanliness': 0, 'food': 3, 'price': 1, 'others': 0}


100%|██████████| 2532/2532 [1:44:11<00:00,  2.47s/it]


Item 2531:
Review: Unique,Andrew, had amazing customer service, and my food came out fresh.
Generated: Analysis:
service=3
cleanliness=0
food=3
price=0
others=0


Parsed sentiments: {'service': 3, 'cleanliness': 0, 'food': 3, 'price': 0, 'others': 0}


In [26]:
# Convert y_pred to a DataFrame
y_pred_df = pd.DataFrame(y_pred)

# Reset index of X_test to ensure it aligns with y_pred_df
df = df.reset_index(drop=True)

# Concatenate X_test with y_pred_df
result_df = pd.concat([df, y_pred_df], axis=1)

In [27]:
# Display the first few rows of the result
result_df.head()

,store_address,location,reviewer_id,review_time,actual_sentiment,review,token_review_clean,service,cleanliness,food,price,others
0,"1044 US-11, Champlain, NY 12919, United States",12919,11931,6 months ago,negative,Food seems to be getting worse. Tables dirty. Sketchy crowd loitering within the store.,food seems getting worse table dirty sketchy crowd loitering within store,1,1,1,0,1
1,"1044 US-11, Champlain, NY 12919, United States",12919,11895,a week ago,negative,Food was slow. Manager (short hair woman) was having a mental breakdown or something and was being rude. She needs to be fired. Her name was Elizabeth or something. Very unprofessional.,food slow manager short hair woman mental breakdown something rude need fired name elizabeth something unprofessional,1,0,1,0,1
2,"1044 US-11, Champlain, NY 12919, United States",12919,11920,a month ago,negative,Franchise owners need to hire more staff to keep up with demand,franchise owner need hire staff keep demand,1,0,0,0,1
3,"1044 US-11, Champlain, NY 12919, United States",12919,11912,2 weeks ago,negative,Haven't seen a McDonald's so lost in my life :(,seen lost life,0,0,0,0,1
4,"1044 US-11, Champlain, NY 12919, United States",12919,11897,4 months ago,negative,"I came there to pick up order around 5 in the morning, there was a manager(brown hair) who dealt with me very rudely, i politely requested for my uber pick up but he showed me attitude and just shut the window. That was an awful experience for me and i would like to suggest them to show some etiquettes and respect towards their customers. I also work in a restaurant and know how to deal with the customers and uber orders.",came pick order around morning managerbrown hair dealt rudely politely requested uber pick showed attitude shut window awful experience would like suggest show etiquette respect towards customer also work restaurant know deal customer uber order,1,0,0,0,1


In [28]:
result_df.to_csv('Labelled_ABSA.csv')